# agex: Code-Native Agents

In this demo, we'll start with a 'wide' dataset of stock prices and use an `agex` agent to perform a classic data science task: reshaping it into a 'long' format and then visualizing the result.

In [1]:
import plotly.express as px

df = px.data.stocks()
display(df.head())

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


First, we create an `Agent`. Instead of rigid "tools", we give our agent direct access to the `pandas` and `plotly` libraries, creating a guided sandbox environment for it to work in.

In [2]:
import pandas as pd
import plotly.graph_objects as go
from agex import Agent

agent = Agent(name="data_expert")

# Register pandas & plotly for the agent
agent.cls(go.Figure, visibility="low")
agent.module(px, visibility="low")
agent.module(pd, visibility="low")

Next, we define the agent's skills using the `@agent.task` decorator. We provide a function signature with type hints, and the agent writes the implementation at runtime. The types are the contract.

In [3]:
@agent.task
def transform_data(prompt: str, df: pd.DataFrame) -> pd.DataFrame:  # type: ignore[return-value]
    """Transform the data according to the prompt."""
    pass

@agent.task
def plot_data(prompt: str, df: pd.DataFrame) -> go.Figure:  # type: ignore[return-value]
    """Plot the data according to the prompt."""
    pass

Let's ask the agent to transform our data. Notice that the agent will return a real, live `DataFrame` directly to us (not JSON).

In [4]:
prompt = "Reshape this data from wide to long with 'date', 'company', and 'price' cols"
long_df = transform_data(prompt, df)
display(long_df.head())

,date,company,price
0,2018-01-01,GOOG,1.000000
1,2018-01-08,GOOG,1.018172
2,2018-01-15,GOOG,1.032008
3,2018-01-22,GOOG,1.066783
4,2018-01-29,GOOG,1.008773


That's a real `DataFrame`, but how did the agent create it? We can see the agent's "thinking" by re-running the task with an `on_event` handler.

In [6]:
from agex import ActionEvent, Event, TaskStartEvent

def display_action(event: Event):
    if isinstance(event, ActionEvent):
        display(event)

prompt = "Reshape this data from wide to long with 'date', 'company', and 'price' cols"
long_df = transform_data(prompt, df, on_event=display_action)


ActionEvent(ActionEvent[data_expert] @ 09:03:08)

Now that the agent has prepared the data, let's ask it to create the final visualization.

In [10]:

prompt = "Create a line chart of the stock prices over time, using a different color for each company."
plot = plot_data(prompt, long_df, on_event=display_action)
display(plot)

ActionEvent(ActionEvent[data_expert] @ 09:19:11)

ActionEvent(ActionEvent[data_expert] @ 09:19:13)

## Conclusion

In this demo, we saw how `agex` agents:

- Use real Python libraries directly.
- Receive and return complex objects like DataFrames and Figures.
- Provide full observability into their thought process.